# Model with Metrics

Example testing a model with custom metrics.

Metrics can be 

  * A ```COUNTER``` : the returned value will increment the current value
  * A ```GAUGE``` : the returned value will overwrite the current value
  * A ```TIMER``` : a number of millisecs. Prometheus SUM and COUNT metrics will be created.
  
You need to provide a list of dictionaries each with the following:

  * a ```type``` : COUNTER, GAUGE, or TIMER
  * a ```key``` : a user defined key
  * a ```value``` : a float value
  
See example code below:
 

In [1]:
!pygmentize ModelWithMetrics.py

class ModelWithMetrics(object):

    def __init__(self):
        print("Initialising")

    def predict(self,X,features_names):
        print("Predict called")
        return X

    def metrics():
        return [
            {"type":"COUNTER","key":"mycounter","value":1}, # a counter which will increase by the given value
            {"type":"GAUGE","key":"mygauge","value":100}, # a gauge which will be set to given value
            {"type":"TIMER","key":"mytimer","value":20.2}, # a timer which will add sum and count metrics - assumed millisecs
            ]
    


# REST

In [1]:
!s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-rest:0.1

---> Installing application source...
Build completed successfully


In [2]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-rest:0.1

672480d40f662cc3d37d999264ea3ecca234eb9bb0cbd685772f1f5a8e515240


In [3]:
!cd ../../../wrappers/testing && make build_protos

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
rm -f fbs/*.py
rm -rf fbs/__pycache__
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [4]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 'ndarray': [[6.668, 4.529, 4.791, 0.649]]}}
RECEIVED RESPONSE:
{'data': {'names': ['t:0', 't:1', 't:2', 't:3'], 'ndarray': [[6.668, 4.529, 4.791, 0.649]]}, 'meta': {'metrics': [{'key': 'mycounter', 'type': 'COUNTER', 'value': 1}, {'key': 'mygauge', 'type': 'GAUGE', 'value': 100}, {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}]}}

Time 0.00577998161315918


In [5]:
!docker rm model-with-metrics --force

model-with-metrics


# gRPC

In [6]:
!s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-grpc:0.1

---> Installing application source...
Build completed successfully


In [7]:
!docker run --name "model-with-metrics" -d --rm -p 5000:5000 model-with-metrics-grpc:0.1

b83028c73787596d2a641c84eedc2173951ad6746d78897e459173ce4799a521


In [8]:
!cd ../../../wrappers/testing && make build_protos

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
rm -f fbs/*.py
rm -rf fbs/__pycache__
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [9]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p --grpc

----------------------------------------
SENDING NEW REQUEST:
data {
  names: "sepal_length"
  names: "sepal_width"
  names: "petal_length"
  names: "petal_width"
  ndarray {
    values {
      list_value {
        values {
          number_value: 5.681
        }
        values {
          number_value: 4.435
        }
        values {
          number_value: 6.005
        }
        values {
          number_value: 0.73
        }
      }
    }
  }
}

RECEIVED RESPONSE:
meta {
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
    value: 20.200000762939453
  }
}
data {
  names: "t:0"
  names: "t:1"
  names: "t:2"
  names: "t:3"
  ndarray {
    values {
      list_value {
        values {
          number_value: 5.681
        }
        values {
          number_value: 4.435
        }
        values {
          number_value: 6.005
        }
        values {
          number_

In [10]:
!docker rm model-with-metrics --force

model-with-metrics


# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need Minikube version 0.25.2**

In [26]:
!minikube start --vm-driver kvm2 --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

There is a newer version of minikube available (v0.30.0).  Download it here:
https://github.com/kubernetes/minikube/releases/tag/v0.30.0

To disable this notification, run the following:
minikube config set WantUpdateNotification false
Starting local Kubernetes v1.9.4 cluster...
Starting VM...
Getting VM IP address...
Moving files into cluster...
Setting up certs...
Connecting to cluster...
Setting up kubeconfig...
Starting cluster components...
Kubectl is now configured to use the cluster.
Loading cached images from config file.


In [27]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

clusterrolebinding.rbac.authorization.k8s.io/kube-system-cluster-admin created


In [28]:
!helm init

$HELM_HOME has been configured at /home/clive/.helm.

Tiller (the Helm server-side component) has been installed into your Kubernetes Cluster.

Please note: by default, Tiller is deployed with an insecure 'allow unauthenticated users' policy.
To prevent this, run `helm init` with the --tiller-tls-verify flag.
For more information on securing your installation see: https://docs.helm.sh/using_helm/#securing-your-helm-installation
Happy Helming!


In [29]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

Waiting for deployment "tiller-deploy" rollout to finish: 0 of 1 updated replicas are available...
deployment "tiller-deploy" successfully rolled out


In [30]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

NAME:   seldon-core-crd
LAST DEPLOYED: Sat Nov  3 08:25:19 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ServiceAccount
NAME                        SECRETS  AGE
seldon-spartakus-volunteer  1        0s

==> v1beta1/ClusterRole
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1beta1/ClusterRoleBinding
NAME                        AGE
seldon-spartakus-volunteer  0s

==> v1/ConfigMap
NAME                     DATA  AGE
seldon-spartakus-config  3     1s

==> v1beta1/CustomResourceDefinition
NAME                                         AGE
seldondeployments.machinelearning.seldon.io  0s

==> v1beta1/Deployment
NAME                        DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
seldon-spartakus-volunteer  1        0        0           0          0s


NOTES:
NOTES: TODO


NAME:   seldon-core
LAST DEPLOYED: Sat Nov  3 08:25:20 2018
NAMESPACE: default
STATUS: DEPLOYED

RESOURCES:
==> v1/ClusterRoleBinding
NAME            AGE
seldon-default  0s

==> v1beta1/Ro

In [11]:
!helm install seldon-core-analytics --name seldon-core-analytics --set grafana_prom_admin_password=password --set persistence.enabled=false --repo https://storage.googleapis.com/seldon-charts 

NAME:   seldon-core-analytics
LAST DEPLOYED: Thu Nov  8 13:40:09 2018
NAMESPACE: seldon
STATUS: DEPLOYED

RESOURCES:
==> v1/Secret
NAME                 TYPE    DATA  AGE
grafana-prom-secret  Opaque  1     0s

==> v1/ConfigMap
NAME                       DATA  AGE
alertmanager-server-conf   1     0s
grafana-import-dashboards  7     0s
prometheus-rules           4     0s
prometheus-server-conf     1     0s

==> v1/ServiceAccount
NAME        SECRETS  AGE
prometheus  1        0s

==> v1beta1/ClusterRole
NAME        AGE
prometheus  0s

==> v1beta1/ClusterRoleBinding
NAME        AGE
prometheus  0s

==> v1/Job
NAME                            DESIRED  SUCCESSFUL  AGE
grafana-prom-import-dashboards  1        0           0s

==> v1beta1/Deployment
NAME                     DESIRED  CURRENT  UP-TO-DATE  AVAILABLE  AGE
alertmanager-deployment  1        1        1           0          0s
grafana-prom-deployment  1        1        1           0          0s
prometheus-deployment    1        1        1 

 * Port forward the dashboard when running
  ```
     kubectl port-forward $(kubectl get pods -n seldon -l app=grafana-prom-server -o jsonpath='{.items[0].metadata.name}') -n seldon 3000:3000
  ```
  * Visit http://localhost:3000/dashboard/db/prediction-analytics?refresh=5s&orgId=1 and login using "admin" and the password you set above when launching with helm.

# REST

In [13]:
!eval $(minikube docker-env) && s2i build -E environment_rest . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-rest:0.1 --loglevel 5

I1108 13:41:46.852606    1559 build.go:50] Running S2I version "v1.1.12"
I1108 13:41:46.852742    1559 util.go:58] Getting docker credentials for seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
I1108 13:41:46.852762    1559 util.go:74] Using  credentials for pulling seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
I1108 13:41:46.882915    1559 docker.go:487] Using locally available image "seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT"
I1108 13:41:46.884452    1559 build.go:163] 
Builder Image:			seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT
Source:				.
Output Image Tag:		model-with-metrics-rest:0.1
Environment:			MODEL_NAME=ModelWithMetrics,API_TYPE=REST,SERVICE_TYPE=MODEL,PERSISTENCE=0
Environment File:		environment_rest
Labels:				
Incremental Build:		disabled
Remove Old Build:		disabled
Builder Pull Policy:		if-not-present
Previous Image Pull Policy:	if-not-present
Quiet:				disabled
Layered Build:			disabled
Docker Endpoint:		tcp://192.168.39.50:2376
Docker Pull Config:		/home/clive/.d

I1108 13:41:47.157389    1559 docker.go:1032] Waiting for container "d661ca52e1d7662ef0badcb2be3688f90759c9756498c0aee3a0089e2a99190b" to stop ...
I1108 13:41:47.244738    1559 docker.go:1057] Invoking PostExecute function
I1108 13:41:47.244769    1559 postexecutorstep.go:68] Skipping step: store previous image
I1108 13:41:47.244773    1559 postexecutorstep.go:117] Executing step: commit image
I1108 13:41:47.246352    1559 postexecutorstep.go:522] Checking for new Labels to apply... 
I1108 13:41:47.246362    1559 postexecutorstep.go:530] Creating the download path '/tmp/s2i141179583/metadata'
I1108 13:41:47.246431    1559 postexecutorstep.go:464] Downloading file "/tmp/.s2i/image_metadata.json"
I1108 13:41:47.284759    1559 postexecutorstep.go:538] unable to download and extract 'image_metadata.json' ... continuing
I1108 13:41:47.287415    1559 docker.go:1091] Committing container with dockerOpts: {Reference:model-with-metrics-rest:0.1 Comment: Author: Changes:[] Pause:false Config:0xc

In [25]:
!kubectl create -f deployment-rest.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [15]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[name:mymodel-mymodel-svc-orch replicas:1 replicasAvailable:1] map[replicasAvailable:1 name:mymodel-mymodel-complex-model-0 replicas:1]] state:Available]

In [16]:
!cd ../../../util/api_tester && make build_protos 

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Test predict

In [37]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

----------------------------------------
SENDING NEW REQUEST:
{'meta': {}, 'data': {'names': ['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], 'ndarray': [[7.974098303518131, 3.149273146516985, 8.219107590512102, 2.3086360840551796]]}}
Getting token from http://192.168.39.50:31307/oauth/token
{"access_token":"343409a6-0abd-4b8c-ad83-dd1f9657fef4","token_type":"bearer","expires_in":42651,"scope":"read write"}
RECEIVED RESPONSE:
{'meta': {'puid': 'l05rsshl95btcv6rvr2k6ig3oa', 'tags': {}, 'routing': {}, 'requestPath': {'complex-model': 'model-with-metrics-rest:0.1'}, 'metrics': [{'key': 'mycounter', 'type': 'COUNTER', 'value': 1.0}, {'key': 'mygauge', 'type': 'GAUGE', 'value': 100.0}, {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}]}, 'data': {'names': ['t:0', 't:1', 't:2', 't:3'], 'ndarray': [[7.974098303518131, 3.149273146516985, 8.219107590512102, 2.3086360840551796]]}}



In [18]:
!kubectl delete -f deployment-rest.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


# gRPC

In [19]:
!eval $(minikube docker-env) && s2i build -E environment_grpc . seldonio/seldon-core-s2i-python3:0.3-SNAPSHOT model-with-metrics-grpc:0.1

---> Installing application source...
Build completed successfully


In [20]:
!kubectl create -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io/mymodel created


Wait until ready (replicas == replicasAvailable)

In [21]:
!kubectl get seldondeployments mymodel -o jsonpath='{.status}' 

map[predictorStatus:[map[replicasAvailable:1 name:mymodel-mymodel-svc-orch replicas:1] map[name:mymodel-mymodel-complex-model-0 replicas:1 replicasAvailable:1]] state:Available]

In [22]:
!cd ../../../util/api_tester && make build_protos 

rm -f proto/prediction*.py
rm -f proto/prediction.proto
rm -rf proto/__pycache__
mkdir -p ./proto
touch ./proto/__init__.py
cp ../../proto/prediction.proto ./proto
python -m grpc.tools.protoc -I. --python_out=. --grpc_python_out=. ./proto/prediction.proto


## Validate on Grafana

To check the metrics have appeared on Prometheus and are available in Grafana you could create a new graph in a dashboard and use the query:

```
mycounter_total
```


## Test predict

In [23]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[1].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p --grpc --oauth-port `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'`

----------------------------------------
SENDING NEW REQUEST:
data {
  names: "sepal_length"
  names: "sepal_width"
  names: "petal_length"
  names: "petal_width"
  ndarray {
    values {
      list_value {
        values {
          number_value: 5.018607555163812
        }
        values {
          number_value: 4.150787936306038
        }
        values {
          number_value: 9.145665131982822
        }
        values {
          number_value: 0.12728175889218607
        }
      }
    }
  }
}

Getting token from http://192.168.39.50:31307/oauth/token
{"access_token":"343409a6-0abd-4b8c-ad83-dd1f9657fef4","token_type":"bearer","expires_in":43122,"scope":"read write"}
RECEIVED RESPONSE:
meta {
  puid: "6k4ba0l998145lnokbc4t2ncff"
  requestPath {
    key: "complex-model"
    value: "model-with-metrics-grpc:0.1"
  }
  metrics {
    key: "mycounter"
    value: 1.0
  }
  metrics {
    key: "mygauge"
    type: GAUGE
    value: 100.0
  }
  metrics {
    key: "mytimer"
    type: TIMER
  

In [24]:
!kubectl delete -f deployment-grpc.json

seldondeployment.machinelearning.seldon.io "mymodel" deleted


In [42]:
!minikube delete

Deleting local Kubernetes cluster...
Machine deleted.
